In [43]:
import pandas as pd
pd.options.display.max_columns = 100

import json
import numpy as np

In [2]:
powerwatch_data = pd.read_csv('s3://wri-public-data/resourcewatch/yale_hackathon/powerwatchdata.tsv', sep='\t')

In [52]:
powerwatch_data[powerwatch_data['capacity_mw'].isnull()].shape

(37, 21)

In [26]:
powerwatch_data.columns

Index(['name', 'pw_idnr', 'capacity_mw', 'year_of_capacity_data', 'country',
       'owner', 'source', 'url', 'latitude', 'longitude', 'commissioning_year',
       'fuel1', 'fuel2', 'fuel3', 'fuel4', 'generation_gwh_2012',
       'generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015',
       'generation_gwh_2016', 'estimated_generation_gwh'],
      dtype='object')

In [75]:
powerwatch_data['fuel1'].unique()

array(['Waste', 'Solar', 'Gas', 'Geothermal', 'Hydro', 'Coal', 'Oil',
       'Wind', 'Biomass', 'Nuclear', 'Other', 'Petcoke', 'Cogeneration',
       'Wave and Tidal', nan], dtype=object)

In [56]:
def prepare_data(df, latcol, loncol, datacol, scenariocol, scenarios):
    '''
    new_dataset_schema = {
        'scenario':'scenario_name',
        'dataseries':['lat', 'lon', 'datapoint', '...']
    }
    '''
    
    json = {"AllData":[]}    
    for scenario in scenarios:
        if scenario == np.nan:
            print('skipping' + scenario)
            continue
            
        newdata = []
        for ix, row in df.iterrows():
            if row[scenariocol]==scenario:
                lat = row[latcol]
                lon = row[loncol]
                data = row[datacol]
                if data == np.nan:
                    print('skipping a row')
                    continue
                newdata.extend([lat, lon, data])
                
        json['AllData'].append({
            "PlantType":str(scenario),
            "Data":newdata
        })
    
    return json
                

In [57]:
kwargs = {
    'df':powerwatch_data,
    'latcol':'latitude',
    'loncol':'longitude',
    'datacol':'capacity_mw',
    'scenariocol':'fuel1',
    'scenarios':powerwatch_data['fuel1'].unique()
}
pwdata4unity = prepare_data(**kwargs)

In [55]:
with open('pwdata4unity.json', 'w') as f:
    f.write(json.dumps(pwdata4unity))

"{'AllData': [{'Year': 'Waste', 'Data': [52.535, 12.3626, 2.5, 42.4803, -87.8861, 7.0, 41.2378, -86.2"

In [ ]:
pwdata4unity = prepare_data(climatewatch_data, 'latitude', 'longitude', 'emissions', 'year', [1860 -- 2015])